## Скачаем наш датасет

In [ ]:
!wget -c http://ai.stanford.edu/~jkrause/car196/car_ims.tgz -O - | tar -xz
!wget -c http://ai.stanford.edu/~jkrause/car196/cars_annos.mat

-                   100%[===================>]   1.82G  20.5MB/s    in 1m 42s  

2021-06-12 10:51:35 (18.2 MB/s) - written to stdout [1956628579/1956628579]

--2021-06-12 10:51:35--  http://ai.stanford.edu/~jkrause/car196/cars_annos.mat
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 394471 (385K) [text/plain]
Saving to: ‘cars_annos.mat’

cars_annos.mat      100%[===================>] 385.23K   408KB/s    in 0.9s    

2021-06-12 10:51:37 (408 KB/s) - ‘cars_annos.mat’ saved [394471/394471]



## Сконвертируем датасет в формат YOLO

In [ ]:
import scipy.io as io
import pandas as pd
import torch
import numpy as np
from PIL import Image
from tqdm import tqdm
import os

fname ='/content/cars_annos.mat'
data  = io.loadmat(fname)

In [ ]:
%cd /content/

path_train = '/content/train/'
path_test = '/content/test/'
path_val = '/content/val/'

labels_path_train = path_train + 'labels/'
labels_path_test = path_test + 'labels/'
labels_path_val = path_val + 'labels/'
img_path_train = path_train + 'images/'
img_path_test = path_test + 'images/'
img_path_val = path_val + 'images/'

for path in [labels_path_train, labels_path_test, 
             img_path_train, img_path_test,
             labels_path_val, img_path_val]:
    os.makedirs(path, exist_ok=True)

def parse_data(data):
    '''Распарсим данные и разложим их в 3 кучки. Train, test и val'''
    for num, i in tqdm(enumerate(range(len(data['annotations'][0])))):
        sample = data['annotations'][0][i]

        img_path = sample[0][0]
        bbox_x1 = sample[1][0][0]
        bbox_y1 = sample[2][0][0]
        bbox_x2 = sample[3][0][0]
        bbox_y2 = sample[4][0][0]
        class_label = sample[5][0][0]-1
        test = sample[6][0][0]

        image = Image.open(img_path)
        width, height = image.size

        label_name = img_path.split('/')[-1].split('.')[0]

        x1 = bbox_x1/width
        x2 = bbox_x2/width
        y1 = bbox_y1/height
        y2 = bbox_y2/height

        bbox_width = x2 - x1
        bbox_height = y2 - y1

        labels_path = labels_path_train
        img_path_final = img_path_train
        if num in np.random.randint(size=(200), low=0, high=len(data['annotations'][0])):
            labels_path = labels_path_val
            img_path_final = img_path_val
        if num in np.random.randint(size=(1000), low=0, high=len(data['annotations'][0])):
            labels_path = labels_path_test
            img_path_final = img_path_test
        with open(labels_path + label_name + '.txt', mode="w") as label_file:
            label_file.write(
                f"{class_label} {x1 + bbox_width / 2} {y1 + bbox_height / 2} {bbox_width} {bbox_height}\n"
            )
            os.rename(img_path, img_path_final + label_name + '.jpg')
        
parse_data(data)

162it [00:00, 1613.24it/s]

/content


16185it [00:07, 2081.51it/s]


In [ ]:
# def class2label(data, indx):
#     return data['class_names'][0][indx-1][0]

In [ ]:
## Создадим файл с настройками для нашей YOLO

In [ ]:
import yaml
%cd /content/
names = np.array([x[0] for x in data['class_names'][0]]).tolist()

setting = dict(
    train = path_train,
    val = path_val,
    test = path_test,
    nc = 196,
    names = names
)

with open('/content/settings.yaml', 'w') as outfile:
    yaml.dump(setting, outfile, default_flow_style=False, )

/content


NameError: ignored

In [ ]:
## Скачаем YOLOv5

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 7103, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 7103 (delta 123), reused 150 (delta 80), pack-reused 6894
Receiving objects: 100% (7103/7103), 9.16 MiB | 20.04 MiB/s, done.
Resolving deltas: 100% (4861/4861), done.
/content/yolov5
     |████████████████████████████████| 645kB 4.1MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
# !pip install wandb # Для мониторинга обучения

In [ ]:
## Затюним нашу предобученную YOLOv5. Будем делать resize изображений, что бы быстрее училась и все влезло в память

In [ ]:
%cd /content/yolov5/
!python train.py --batch 48 --weights yolov5l.pt --data /content/settings.yaml --epochs 50 --cache --img 256 

/content/yolov5
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-180-ge8c5237 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Namespace(adam=False, artifact_alias='latest', batch_size=48, bbox_interval=-1, bucket='', cache_images=True, cfg='', data='/content/settings.yaml', device='', entity=None, epochs=50, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[256, 256], label_smoothing=0.0, linear_lr=False, local_rank=-1, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp3', save_period=-1, single_cls=False, sync_bn=False, total_batch_size=48, upload_dataset=False, weights='yolov5l.pt', workers=8, world_size=1)
tensorboard: Start with 'tensorboard --logdir runs/train', view at http://localhost:6006/
2021-06-11 17:07:52.930688: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Su

## Посчитаем точность на тестовой выборке

In [ ]:
import yaml
%cd /content/
names = np.array([x[0] for x in data['class_names'][0]]).tolist()

setting = dict(
    train = path_train,
    val = path_test,
    nc = 196,
    names = names
)

with open('/content/settings_test.yaml', 'w') as outfile:
    yaml.dump(setting, outfile, default_flow_style=False, )

/content


In [ ]:
trained_weights = '/content/yolov5/runs/train/exp3/weights/best.pt'

In [ ]:
%cd /content/yolov5/
!python test.py --batch 48 --data /content/settings_test.yaml --weights $trained_weights --conf 0.25 --img 256

/content/yolov5
Namespace(augment=False, batch_size=48, conf_thres=0.25, data='/content/settings_test.yaml', device='', exist_ok=False, half=False, imgsz=256, iou_thres=0.6, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='val', verbose=False, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'])
YOLOv5 🚀 v5.0-180-ge8c5237 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 392 layers, 47650641 parameters, 0 gradients
val: Scanning '/content/test/labels.cache' images and labels... 985 found, 0 missing, 0 empty, 0 corrupted: 100% 985/985 [00:00<00:00, 9140242.12it/s]
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 21/21 [00:11<00:00,  1.77it/s]
                 all        985        985      0.771      0.856      0.866      0.817
Speed: 0.1ms pre-process, 5.2ms inference, 1.4ms NMS per image at shape (48, 3, 256, 256)
Results save

## Будем предсказывать наши машинки

In [ ]:
url = 'https://www.supercars.net/blog/wp-content/uploads/2016/04/2012_AstonMartin_V8VantageRoadster1.jpg'
!wget $url 

--2021-06-11 18:22:57--  https://www.supercars.net/blog/wp-content/uploads/2016/04/2012_AstonMartin_V8VantageRoadster1.jpg
Resolving www.supercars.net (www.supercars.net)... 172.67.134.254, 104.21.6.157, 2606:4700:3030::ac43:86fe, ...
Connecting to www.supercars.net (www.supercars.net)|172.67.134.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254280 (248K) [image/jpeg]
Saving to: ‘2012_AstonMartin_V8VantageRoadster1.jpg’

2012_AstonMartin_V8 100%[===================>] 248.32K  --.-KB/s    in 0.03s   

2021-06-11 18:22:57 (9.05 MB/s) - ‘2012_AstonMartin_V8VantageRoadster1.jpg’ saved [254280/254280]



In [ ]:
!python detect.py --source /content/yolov5/2012_AstonMartin_V8VantageRoadster1.jpg --weights $trained_weights --img 256 --max-det 1

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, imgsz=256, iou_thres=0.45, line_thickness=3, max_det=1, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/yolov5/2012_AstonMartin_V8VantageRoadster1.jpg', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'])
YOLOv5 🚀 v5.0-180-ge8c5237 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 392 layers, 47650641 parameters, 0 gradients
image 1/1 /content/yolov5/2012_AstonMartin_V8VantageRoadster1.jpg: 224x256 1 Aston Martin V8 Vantage Convertible 2012, Done. (0.021s)
Results saved to runs/detect/exp4
Done. (0.088s)


In [ ]:
url2 = 'https://i.pinimg.com/originals/b6/9c/b2/b69cb232b50b1226a9245e19b0e4d455.jpg'
!wget $url2

--2021-06-11 18:33:22--  https://images.drive.ru/i/0/5ecf4bfeec05c49473000003.jpg
Resolving images.drive.ru (images.drive.ru)... 146.255.192.80, 146.255.192.81
Connecting to images.drive.ru (images.drive.ru)|146.255.192.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [image/jpeg]
Saving to: ‘5ecf4bfeec05c49473000003.jpg’

5ecf4bfeec05c494730     [  <=>               ] 117.72K   215KB/s    in 0.5s    

2021-06-11 18:33:24 (215 KB/s) - ‘5ecf4bfeec05c49473000003.jpg’ saved [120548]



In [ ]:
!python detect.py --source /content/yolov5/b69cb232b50b1226a9245e19b0e4d455.jpg --weights $trained_weights --img 256 --max-det 1

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, imgsz=256, iou_thres=0.45, line_thickness=3, max_det=1, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/yolov5/b69cb232b50b1226a9245e19b0e4d455.jpg', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'])
YOLOv5 🚀 v5.0-180-ge8c5237 torch 1.8.1+cu101 CUDA:0 (Tesla T4, 15109.75MB)

Fusing layers... 
Model Summary: 392 layers, 47650641 parameters, 0 gradients
image 1/1 /content/yolov5/b69cb232b50b1226a9245e19b0e4d455.jpg: 192x256 1 Eagle Talon Hatchback 1998, Done. (0.021s)
Results saved to runs/detect/exp6
Done. (0.070s)


In [ ]:
!python detect.py --source /content/val/images --weights /content/yolov5/runs/train/exp3/weights/best.pt --img 256 --max-det 1

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, imgsz=256, iou_thres=0.45, line_thickness=3, max_det=1, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/val/images', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'])
YOLOv5 🚀 v5.0-181-g4984cf5 torch 1.8.1+cu101 CPU

Traceback (most recent call last):
  File "detect.py", line 204, in <module>
    detect(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 56, in detect
    model = attempt_load(weights, map_location=device)  # load FP32 model
  File "/content/yolov5/models/experimental.py", line 119, in attempt_load
    ckpt = torch.load(attempt_download(w), map_location=map_location)  # load
  File "/usr/loca

In [ ]:
url3 = "https://www.bmw.ru/content/dam/bmw/common/all-models/3-series/sedan/2018/navigation/bmw-3-series-modellfinder.png/_jcr_content/renditions/cq5dam.resized.img.585.low.time1579021049723.png"
!wget $url3

!python detect.py --source /content/yolov5/cq5dam.resized.img.585.low.time1579021049723.png --weights /content/yolov5/runs/train/exp3/weights/best.pt --img 256 --max-det 1

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.25, device='', exist_ok=False, half=False, hide_conf=False, hide_labels=False, imgsz=256, iou_thres=0.45, line_thickness=3, max_det=1, name='exp', nosave=False, project='runs/detect', save_conf=False, save_crop=False, save_txt=False, source='/content/yolov5/cq5dam.resized.img.585.low.time1579021049723.png', update=False, view_img=False, weights=['/content/yolov5/runs/train/exp3/weights/best.pt'])
YOLOv5 🚀 v5.0-181-g4984cf5 torch 1.8.1+cu101 CPU

Traceback (most recent call last):
  File "detect.py", line 204, in <module>
    detect(**vars(opt))
  File "/usr/local/lib/python3.7/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "detect.py", line 56, in detect
    model = attempt_load(weights, map_location=device)  # load FP32 model
  File "/content/yolov5/models/experimental.py", line 119, in attempt_load
    ckpt = torch.load(attempt_download(w), map_loc